# One big biome table

Since I am hoping to make comparisons of wood and leaf endophyte environmental patterns, I need to combine these datasets early in the biomformatics pipeline, to make them as comparable as possible. We'll try to stick to the [usearch (uparse)](http://drive5.com/usearch/) pipeline for the process, as much as possible.

<a id='environment'></a>

### Work environment

Working directory, on my machine:

In [2]:
cd /home/daniel/Documents/Taiwan_data/combined/combo_biome

We'll be using the [usearch (uparse)](http://drive5.com/usearch/) pipeline, version usearch v8.1.1861_i86linux32 on my personal machine, and the equivalent 64-bit version on our computing cluster [ACISS](http://aciss-computing.uoregon.edu/). These programs are abbreviated (soft linked) to "usearch81" and "usearch", respectively. 

<a id='mergepairs'></a> 

### Merging paired-end reads

Let's re-pair all readsets in the same manner, except for Roo's stromatal readset, which he aligned by hand. First we trim a little to make sure we doing our alignments with high quality base calls. The sites for trimming are decided by looking at the raw reads [(see below)](#quality), and finding where quality begins to drop off. 
To trim, we'll use the [FASTX-toolkit](http://hannonlab.cshl.edu/fastx_toolkit/).

In [17]:
## wood
fastx_trimmer -l 255 -i $wdir/woodR1.fastq -o woodR1_trimmed.fastq
fastx_trimmer -l 210 -i $wdir/woodR2.fastq -o woodR2_trimmed.fastq

## leaves. these sites come from Roo's analysis.
fastx_trimmer -l 263 -i $roo/Roo_R1.fastq -o Roo_R1_trimmed.fastq
fastx_trimmer -l 170 -i $roo/Roo_R2.fastq -o Roo_R2_trimmed.fastq

Do the actual pairing:

In [4]:
## wood, let's pair both trimmed and untrimmed just to compare:
usearch -fastq_mergepairs /home6/dthomas/combobiom/woodR1_trimmed.fastq -reverse /home6/dthomas/combobiom/woodR2_trimmed.fastq -fastqout woodtrimmedmerged.fastq -report report.txt
usearch -fastq_mergepairs /home6/dthomas/combobiom/woodR1.fastq -reverse /home6/dthomas/combobiom/woodR2.fastq -fastqout woodmerged.fastq -report report.txt

In [5]:
## leaves
usearch -fastq_mergepairs Roo_R2.fastq -reverse Roo_R1.fastq -fastqout leafmerged.fastq
usearch -fastq_mergepairs Roo_R2_trimmed.fastq -reverse Roo_R1_trimmed.fastq -fastqout leaftrimmedmerged.fastq

What did trimming do to our reads? Let's take a look. To plot these we'll use the [fastx wrapper](http://hannonlab.cshl.edu/fastx_toolkit/commandline.html#fastq_quality_boxplot_usage) for [gnuplot](http://www.gnuplot.info/), which I've tinkered with just a little to change up the crowded axes of the original setup. Fastx requires that we first compile the quality data from the fastq files:

In [18]:
## wood quality stats:
fastx_quality_stats -i woodR1.fastq -o woodR1_fastxstats.txt
fastx_quality_stats -i woodR2.fastq -o woodR2_fastxstats.txt
fastx_quality_stats -i woodR1_trimmed.fastq -o woodR1_fastxstats.txt
fastx_quality_stats -i woodR2_trimmed.fastq -o woodR2_fastxstats.txt
fastx_quality_stats -i woodmerged.fastq -o woodmerged_fastxstats.txt
fastx_quality_stats -i woodtrimmedmerged.faestq -o woodtrimmedmerged_fastxstats.txt

In [ ]:
## leaf quality stats:
fastx_quality_stats -i Roo_R1.fastq -o Roo_R1_fastxstats.txt
fastx_quality_stats -i Roo_R2.fastq -o Roo_R2_fastxstats.txt
fastx_quality_stats -i Roo_R1_trimmed.fastq -o Roo_R1_trimmed_fastxstats.txt
fastx_quality_stats -i Roo_R2_trimmed.fastq -o Roo_R2_trimmed_fastxstats.txt
fastx_quality_stats -i leafmerged.fastq -o leafmerged_fastxstats.txt


Both of thoses took just minutes to due on ACISS. How do they look? First, line counts

In [14]:
## our leaf reads are kept in a separate folder:
roo='/home/daniel/Documents/Taiwan_data/leaves/roo_illumina' 
wc -l $roo/Roo_R1.fastq
wc -l $roo/Roo_R2.fastq

72806460 /home/daniel/Documents/Taiwan_data/leaves/roo_illumina/Roo_R1.fastq
72806460 /home/daniel/Documents/Taiwan_data/leaves/roo_illumina/Roo_R2.fastq


In [7]:
wc -l leafmerged.fastq
wc -l leaftrimmedmerged.fastq

42133528 leafmerged.fastq


And wood:

In [8]:
woodR='/home/daniel/Documents/Taiwan_data/wood/woodbiom' 
wc -l $woodR/woodR1.fastq 
wc -l $woodR/woodR2.fastq 
wc -l woodmerged.fastq 

22271196 /home/daniel/Documents/Taiwan_data/wood/woodbiom/woodR1.fastq
22271196 /home/daniel/Documents/Taiwan_data/wood/woodbiom/woodR2.fastq
21293096 woodmerged.fastq


Make the graphics:

In [20]:
## wood graphics
./dan_plot.sh -i woodR1_fastxstats.txt -o woodR1_quality.png
./dan_plot.sh -i woodR2_fastxstats.txt -o woodR2_quality.png
./dan_plot.sh -i  woodR1_trimmed_fastxstats.txt -o woodR1_trimmed_quality.png
./dan_plot.sh -i  woodR2_trimmed_fastxstats.txt -o woodR2_trimmed_quality.png
./dan_plot.sh -i woodmerged_fastxstats.txt -o woodmerged_quality.png
./dan_plot.sh -i  woodtrimmedmerged_fastxstats.txt -o woodtrimmedmerged_quality.png

## leaf graphics
./dan_plot.sh -i  Roo_R1_trimmed_fastxstats.txt -o Roo_R1_trimmed_quality.png
./dan_plot.sh -i  Roo_R2_trimmed_fastxstats.txt -o Roo_R2_trimmed_quality.png
./dan_plot.sh -i leaftrimmedmerged_fastxstats.txt -o leaftrimmedmerged_quality.png

<a id='quality'></a>

The **untrimmed wood R1** reads look like this:

<img src='woodR1_quality.png'>

Compare to **trimmed wood R1** reads:

<img src='woodR1_trimmed_quality.png'>

The **untrimmed wood R2** reads look like this:

<img src='woodR2_quality.png'>

Compare to **trimmed wood R2** reads:

<img src='woodR2_trimmed_quality.png'>

And the **untrimmed, merged wood** file looks like this:

<img src='woodmerged_quality.png'>

Obviously some problems here. So compare to the **merged, trimmed wood** reads:

<img src='woodtrimmedmerged_quality.png'>

Looks much better, but still a large dip in quality around 15 bp. More on this later.

Roo has already decided the trimming sites for his leaf data [(see above)](#mergepairs). Skip to the trimmed leaf R1 reads:

<img src='Roo_R1_trimmed_quality.png'>

The trimmed leaf R2 reads:

<img src='Roo_R2_trimmed_quality.png'>

Trimmed, merged leaf reads:

<img src='leaftrimmedmerged_quality.png'>